In [1]:
import glob
import pandas as pd

In [2]:
def add_date(file_):
    df_ = pd.read_csv(file_, header=None, low_memory=False)
    df_.columns = ["file_id","customer_id","queryts","product_id"]
    df_["month"] = file_[17:19]
    df_["date"] = file_[19:21]

    df_.month = pd.to_numeric(df_.month, errors = 'coerce')
    df_.date = pd.to_numeric(df_.date, errors = 'coerce')

    month = file_[18:19]
    if month == str(3):
        df_["date_new"] = df_.date
    elif month == str(4):
        df_["date_new"] = df_.date + 31
    else:
        df_["date_new"] = df_.date + 31 + 30

    return df_

In [3]:
for path_ in ['train1','train2','train3','test_1','test_2','test_3']:
    allFiles = glob.glob("query_log/" + path_ + "/*.csv")

    df = pd.DataFrame()
    for file_ in allFiles:
        df_ = add_date(file_)
        df = pd.concat([df, df_], axis=0)

    df.to_pickle("dataset/" + path_ + ".pkl")
    print (df.shape)

(6898481, 7)
(17966156, 7)
(21775315, 7)
(11558635, 7)
(21475256, 7)
(3599267, 7)


In [4]:
df_train_set = pd.read_csv("dataset/training-set.csv")
df_test_set = pd.read_csv("dataset/testing-set.csv")

df_train_set["type"] = 'train'
df_test_set["type"] = 'test'

df_set = pd.concat([df_train_set, df_test_set], axis=0)

print ("number of file :", len(set(df_set.file_id)))
print (df_set.type.value_counts())

number of file : 81977
train    52559
test     29418
Name: type, dtype: int64


In [5]:
df = pd.read_pickle("dataset/test_1.pkl")
df = pd.merge(df, df_set, how='left', on='file_id')

In [6]:
df1 = df[(df.type == "train")]
df2 = df[(df.type == "test")]

print (df.type.value_counts())
print (df1.shape,df2.shape)

train    7610293
test     3948342
Name: type, dtype: int64
(7610293, 9) (3948342, 9)


In [7]:
df1 = df1.drop(["label", "type"], axis=1)
df2 = df2.drop(["label", "type"], axis=1)

In [8]:
df1.to_pickle("dataset/valid.pkl")
df2.to_pickle("dataset/test_1.pkl")

In [9]:
df_.iloc[0, :]

file_id        6c9489436c2148002f069f595f9b197d
customer_id    64be4c0b64acd1012827dc87c9e13bca
queryts                              1496016000
product_id                               c76d58
month                                         5
date                                         29
date_new                                     90
Name: 0, dtype: object

In [10]:
df2.iloc[0, :]

file_id        9c1e66830d290960ef5f31d9b7a9be19
customer_id    a3814a821245b96665fcad048f2d8fe5
queryts                              1494288010
product_id                               c105a0
month                                         5
date                                          9
date_new                                     70
Name: 0, dtype: object

> clear